In [252]:
# Import the libraries
#!pip install openpyxl
import pandas as pd
from IPython.display import display, HTML
# Set the display width to control the output width
pd.set_option('display.width', 100)
# Read the dataset and read the Laptop Dataset
df = pd.read_csv('Indian_food_with_full_description.csv')
df

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region,description
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East,Balu shahi is a vegetarian dessert from West B...
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West,Boondi is a vegetarian dessert from Rajasthan ...
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North,Gajar ka halwa is a vegetarian dessert from Pu...
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West,Ghevar is a vegetarian dessert from Rajasthan ...
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East,Gulab jamun is a vegetarian dessert from West ...
...,...,...,...,...,...,...,...,...,...,...
250,Til Pitha,"Glutinous rice, black sesame seeds, gur",vegetarian,5,30,sweet,dessert,Assam,North East,Til Pitha is a vegetarian dessert from Assam i...
251,Bebinca,"Coconut milk, egg yolks, clarified butter, all...",vegetarian,20,60,sweet,dessert,Goa,West,Bebinca is a vegetarian dessert from Goa in th...
252,Shufta,"Cottage cheese, dry dates, dried rose petals, ...",vegetarian,-1,-1,sweet,dessert,Jammu & Kashmir,North,Shufta is a vegetarian dessert from Jammu & Ka...
253,Mawa Bati,"Milk powder, dry fruits, arrowroot powder, all...",vegetarian,20,45,sweet,dessert,Madhya Pradesh,Central,Mawa Bati is a vegetarian dessert from Madhya ...


In [253]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            255 non-null    object
 1   ingredients     255 non-null    object
 2   diet            255 non-null    object
 3   prep_time       255 non-null    int64 
 4   cook_time       255 non-null    int64 
 5   flavor_profile  255 non-null    object
 6   course          255 non-null    object
 7   state           255 non-null    object
 8   region          254 non-null    object
 9   description     255 non-null    object
dtypes: int64(2), object(8)
memory usage: 20.0+ KB


##### BUilding a Food recipe recommendation Chatbot #####

Now let's go ahead and understand the system design for the chatbot.


`Stage 1`:
- Intent clarity.
- Intent Confirmation. 

`Stage 2`:
- Product Mapping Layer. 
- Product Information Extraction Layer. 

`Stage 3`:
- Product Recommendation Layer. 


##### Major functions behind the Chatbot

Let's now look at a brief overview of the major functions that form the chatbot. We'll take a deep dive later

- `initialize_conversation()`: This initializes the variable conversation with the system message.
- `get_chat_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant
- `moderation_check()`: This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, it ends the conversation.
- `intent_confirmation_layer()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not GPU intensity, Display quality, Portability, Multitasking, Processing speed, Budget
- `dictionary_present()`: This function checks if the final understanding of user's profile is returned by the chatbot as a python dictionary or not. If there is a dictionary, it extracts the information as a Python dictionary.
- `compare_laptops_with_user()`: This function compares the user's profile with the different laptops and come back with the top 3 recommendations.
- `initialize_conv_reco()`: Initializes the recommendations conversation

In [254]:
# Import the libraries
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [255]:
openai.api_key = open("../MamathaK_Key.txt",'r').read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [424]:
def initialize_conversation(data):
    '''Return's a list [{"role":"system", "content": system_message}]'''
    df=data.copy()
    # Example references (keep them short so system prompt is not huge)
    dish_names = df['name'].unique().tolist()[:20]   # sample first 20 dish names
    dataset_preview = df.head(5).to_dict(orient="records")  # show first 5 rows as dicts

    delimiter = "#####"
    recipe_ghevar = f"""Step 1 Melt ghee and add ice cubes in it to purify it To prepare this authentic Rajasthani delicacy,
                  heat some ghee in a heavy-bottomed pan and once the ghee is hot enough, turn off the flame and add ice cubes. 
        This will extract the impurities and only pure ghee will remain in the pan.
        Step 2 Prepare the Ghevar batter and fry in ghee Make sure you beat the ghee and ice well until the texture turns foamy.
        Now add flour to the ghee and prepare the batter. 
        Heat ghee in a heavy-bottomed pan, place a circular mould and start pouring the batter in centre, fry till it is golden brown.
        Step 3 Prepare the sugar syrup for the Ghevar Now, to prepare the sugar syrup for the main dish, 
        heat water and sugar till the sugar dissolves completely. Add the fried Ghevar to the warm syrup and remove.
        Step 4 Prepare the Malai Ghevar with fresh malai To make the malai, heat milk, sugar,
        cardamom powder, saffron and reduce to half. Allow the malai coating to settle down and thicken.
        Top the malai on the ghevar and garnish with chopped nuts and saffron strands"""

    ingredient_ghevar = f""" For Ghevar Batter : 1 cup all-purpose flour (maida), ¼ cup ghee (clarified butter) chilled, ½ cup chilled milk
    1½ to 2 cups chilled water (as needed),Ice cubes (for mixing).
    For Sugar Syrup:1 cup sugar ,½ cup water, 2–3 cardamom pods crushed, Few strands saffron (optional)
    For Garnish: Chopped pistachios & almonds ,Silver leaf (varq) (optional), Rabri (for special ghevar) """
    
    recipe_laddu = f"""Method:Roast the flour, Heat a heavy-bottomed pan and add the ghee.
        Add besan and roast on low flame, stirring continuously.
        Roast until it turns golden brown and you get a nutty aroma (8–10 minutes). Add cardamom & nuts
        Add chopped nuts and raisins, roast for 1–2 minutes. Turn off the heat and let the mixture cool slightly.
        Add sugar, Once warm (not hot), mix in powdered sugar and cardamom powder.Stir well until evenly combined.Shape laddus
        Take small portions of the mixture in your palm and shape them into round laddus. Cool & store. 
        Let them set for 15–20 minutes. Store in an airtight container."""

    ingredient_laddu = f""" 2 cups besan (gram flour), 1 cup powdered sugar,1/2 cup ghee (clarified butter),
    4–5 cardamom pods (powdered),8–10 cashews or almonds (chopped, optional), 1 tbsp raisins (optional)"""
    
    example_user_dict2 = {
        'name' : 'laddu',
        'ingredients': ingredient_laddu,
        'diet':'vegetarian',
        'prep_time': 10,
        'cook_time':40,
        'course':'desert',
        'flavor_profile':'sweet',
        'state': '-',
        'region':'-',
        'method': recipe_laddu
    }
    
    example_user_dict={
        'name' : 'Ghevar',
        'ingredients': ingredient_ghevar,
        'diet' : 'vegetarian',
        'prep_time': 15,
        'cook_time':  30,
        'flavor_profile': 'sweet',
        'course': 'dessert',
        'state': 'Rajasthan',
        'region': 'West',
        'method': recipe_ghevar
    }
    user_req = {
        "ingredients": "carrots",
        "diet":"vegetarian",
        "prep_time":15,
        "cook_time": 60,
        "flavor_profile": "Sweet",
        "course": "Dessert",
        "state": "Punjab",
        "region": "North",
    }    
    example_user_req={ 'name' : "-",
        'ingredients': "-",
        'diet' : '-',
        'prep_time': '-',
        'cook_time':  '-',
        'flavor_profile': '-',
        'course': '-',
        'state': '-',
        'region': '-',
        'method': '-'
        }
    sample ={
                "ingredients": "jaggery",
                "diet" : "vegetarian",
                "prep_time" : "10 mins",
                "cook_time" :  "15 mins",
                "flavor_profile": "sweet",
                "course" : "starter",
                "state" : "Rajasthan",
                "region" : "South",
                }

    
    system_message = f"""
        You are an intelligent cooking assistant who will suggest strictly Indian Food Recipes for end users.
        You will strictly reject any other queries from endusers and prompt again for request related to Indian Food recipes.
        {delimiter}

        You will get requirement from user input.
        Input can be of two types
            1. User will input dishname name directly. In this case cooking assistant will check if dishname exists in dataframe df in column 'name'.
                if Yes, then only it will get recipe from Internet and update it in 'method' key of python dictionary. 
                Remaining keys "ingredients",'cook_time','prep_time','diet','flavor_profile','course','region','state' values will be obtained from dataframe df row where dishname is found in column name 'name'.
                Close request. 
                example of one such output expected: {example_user_dict}
            2. Is the user asks cooking assistant for suggestion or recipe with specific keys, then cooking assistant will ask for series of questions until 
                cooking assistant gets all the required information. Cooking assistant will get information about fields diet,prep_time,cook_time,flavor_profile,course,state,region,ingredients
                by asking series of questions and map these into python dictionary. Compulsorily and must ask questions until you extract values for all keys diet,prep_time,cook_time,flavor_profile,course,state,region,ingredients
                Example of python dictionary should look something like {user_req}. 
                There will not be "method" key in python dictionary. Output the python dictionary for further actions.
                example of one such food recipe dictionary is {user_req} 

 
        {delimiter}

        Thought 1: Ask the questions to understand the  user requirements. 
        Ask follow-up questions to provide {example_user_req} key values (except for method key) if you are not able to extract. 
        Remember the conditions to ask questions to fill the key values for ["ingredients",'cook_time','prep_time','diet','flavor_profile','course','region','state'] , ask relevant questions if unclear or values doesnt meet the conditions
        mentioned above and fill values from users input.
        acceptable inputs for each key is as follows:
        "ingredients" : <Can be list of grocery items single or multiple. If information is provided in single string convert it to meaning list of ingredients.>
        "cook_time" : <Remember cook_time can be "cook time" or "cooking time" or "time taken to cook". eg: 1) 15 mins, 2) 15 ,3) 15 minutes 4) ten minutes>
        "prep_time" : <prep_time is "preparation_time" or "prep time" or "time taken to prepare". eg: 1) 15 mins, 2) 15 ,3) 15 minutes 4) ten minutes>
        "diet" : <vegetarian|non-vegatarian>
        "course": <main course|snack|starter|dessert>
        "flavor_profile" : <sweet|savory|spicy|bitter>
        "region":<North|South|East|West>
        "state":<Any state part of country India>
        Rephrase the question to capture their necessary information clearly in python dictionary. \n
        {delimiter}
        Thought 2: Now you are trying to fill the values for rest of the keys which you didnt get information from previous step. 
        Remember the instructions set for values for different keys. 
        Ask questions on remaining fields/keys to get their values to strengthen your understanding of the user's profile.
        
        If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
        It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.
        {delimiter}
        Thought 3: Check if you have correctly updated the values for all keys - diet,prep_time,cook_time,flavor_profile,course,state,region,ingredients inside dictionary. 
        It should have values for all keys. '-' is not value hence ask question to get right value for keys having value '-'. 
        Revalidate if all fields/keys are updated compulsorily.
        {delimiter}
        
        Start with small conversation and encourage user to share the requirements
        If user mentions direct dish name and the dish name exists in 'name' column in dataframe df then only obtain required information from dataset and method from Internet.
        else reject saying dish name not found in dataset. 
        if users mentions other fields and not dish name then, go step by step to get details for fields ["diet","prep_time","cook_time","flavor_profile","course","state","region","ingredients"]
        and map it to python dictionary. "method" will not be set strictly.

        {delimiter}
        Here is the sample conversation between user and assistant.
        Sample 1:
        Assistant: Great ! I'm an expert in suggesting Indian recipes. Do you have any specific dish in mind. I can suggest recipe on how to prepare it ??
        User : Sure, I want to prepare Laddu
        Assistant: Great Choice!! Let me grab recipe for preparing laddu. Here is the recipe for laddu.
        Assistant : {example_user_dict2}
        {delimiter}
        Sample 2:
        Assistant: Hi, i'm cooking assistant how can i help you ?
        User : Hi i need recipe that requires preparation time less than 10 mins and cooking time 15 mins and dish should from Rajasthan. It should be starter.
        Assistant : Great ! i'm am a expert in Suggesting Indian recipes for cooking I see want a recipe from Rajasthan with prep_time 10 mins and cooking_time for 15 mins.Should the dessert be sweet or savory ??
        User: Sweet
        Assistant : Should it be vegetarian or non-vegetarian ?
        User: vegetarian. 
        Assistant: Any ingredients to use in mind 
        User: Jaggery
        Assistant : Sure., dessert from which region would like to prefer east, south, west ot north ??
        User: South.
        Assistant : {sample}
        {delimiter}
        Sample 3:
        User: Hi i need recipe of karjikayi
        Assistant: Great! let me check if this information is available in my dataset.
                  Sorry, I dont have information enabled for Karjikayi in my dataset. Please check internet to get recipe for karjikayi.
        {delimiter}

        """
    conversation = [{"role" : "system","content": system_message }]
    return(conversation)
    

In [425]:
debug_conversation = initialize_conversation(df)   
print(debug_conversation)

[{'role': 'system', 'content': '\n        You are an intelligent cooking assistant who will suggest strictly Indian Food Recipes for end users.\n        You will strictly reject any other queries from endusers and prompt again for request related to Indian Food recipes.\n        #####\n\n        You will get requirement from user input.\n        Input can be of two types\n            1. User will input dishname name directly. In this case cooking assistant will check if dishname exists in dataframe df in column \'name\'.\n                if Yes, then only it will get recipe from Internet and update it in \'method\' key of python dictionary. \n                Remaining keys "ingredients",\'cook_time\',\'prep_time\',\'diet\',\'flavor_profile\',\'course\',\'region\',\'state\' values will be obtained from dataframe df row where dishname is found in column name \'name\'.\n                Close request. \n                example of one such output expected: {\'name\': \'Ghevar\', \'ingredien

In [426]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))

def get_chat_completions(input,json_format=False):
    MODEL = "gpt-3.5-turbo"
    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    if(json_format == True):
        input[0]['content'] += system_message_json_output

        chat_completion_json= openai.chat.completions.create(
            model=MODEL,
            response_format={"type":"json_object"},
            seed=1234,
            messages=input
        )
        output = json.loads(chat_completion_json.choices[0].message.content)
    else:
        chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages=input,
            seed=1234,
        )
        output=chat_completion.choices[0].message.content

    return(output)

In [427]:
# Testing the OpenAI functions defined above
input_prompt ='What is the capital of France?'
messages = [{'role':'user','content':input_prompt}]
# system_message_json_output = """<<. Return output in JSON format.>>"""
# messages[0]['content']+=system_message_json_output
messages

[{'role': 'user', 'content': 'What is the capital of France?'}]

In [428]:
get_chat_completions(messages) ## Chat Completions API

'The capital of France is Paris.'

In [429]:
## Get LLM Outputs - JSON output
get_chat_completions(messages, json_format = True) ## Chat Completions API and return in JSON format

{'output': 'Paris'}

##### Debug user input and assistant response

In [430]:
debug_conversation = initialize_conversation(df)
debug_user_input = "HI, I need recipe taht has prep_Time 10 mins and cooking time 15 mins max and it should be savory"
debug_conversation.append({"role":"user","content":debug_user_input})
print(debug_conversation[1]["content"])

HI, I need recipe taht has prep_Time 10 mins and cooking time 15 mins max and it should be savory


In [431]:
debug_response_assistant=get_chat_completions(debug_conversation)
print(debug_response_assistant)

Great! I am an expert in suggesting Indian recipes for cooking. I see you want a recipe with a preparation time of 10 mins and cooking time of 15 mins max. Should the dish be vegetarian or non-vegetarian?


In [432]:
debug_conversation.append({"role":"assistant","content":debug_response_assistant})

##### Moderation Check :
- This layer is required to check if user is asking in appropriate question. If yes the model should end  the conversation

In [433]:
def moderation_layer(user_input):
    response = openai.moderations.create(input=user_input)
    moderation_output=response.results[0].flagged
    if(response.results[0].flagged == True):
        return("Flagged")
    else:
        return("Not Flagged")

In [434]:
op=moderation_layer("I want kill you")
print(op)

Flagged


In [435]:
op=moderation_layer("You are good")
print(op)

Not Flagged


##### Function Intent Confirmation Layer
- Intent confirmation layer required to make sure all the required information is extracted for providing recipes
- fields name, prep_time,cooking_time, region, state, diet,flavor_profile, course are extracted to get tailored response

In [574]:
def intent_confirmation_layer(response_assistant):
    """ The function will check if all the required information are gathered from user input. 
    if there is missing of information , please make sure to ask relevant question to gather information needed. """
    MODEL= "gpt-3.5-turbo"
    delimiter = "####"
    prompt = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 8 keys.
    You are provided an input. The order of keys is not important. User can provide requirement in different order. But all the keys should be updated with values compulsorily. You need to evaluate and map correct values to its respective keys:
        {{ 
        ingredients: <string>,
        diet: <vegetarian|non-vegetarian>,
        prep_time: <number>,
        cook_time: <number>,
        flavor_profile: <sweet|bitter|spicy|savoury>,
        state: <State in India>,
        course: <main course|snack|starter|dessert>,
        region: <North|South|East|West>
    }}
    Next you need to evaluate if the keys have the the values filled correctly.
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys ['ingredients','prep_time','cook_time','diet','flavor_profile','course','region','state'] , Print No as output if not.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'. List of keys not having values. Ignore 'recipe','name','description' keys strictly.
    THought 3 - Think carefully before the answering. 
    """
    message=[{"role": "system", "content":prompt },
              {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=message,
        seed=1234
    )
    output=json.loads(response.choices[0].message.content)
    #print(response)
    return(output)


    

In [508]:
Gdebug_response_intent_layer =  initialize_conversation(df)
debug_response_intent_layer.append({"role":"user","content":"I need starter recipe that can take 15 mins prep_Time and 30 min cook_time. It should be non-vegetarian and it should be from Kerala"})
response=get_chat_completions(debug_response_intent_layer)
print(response)
print(debug_response_intent_layer)

Great! I can help you with that. Should the starter be spicy, savory, sweet, or bitter?
[{'role': 'system', 'content': '\n        You are an intelligent cooking assistant who will suggest strictly Indian Food Recipes for end users.\n        You will strictly reject any other queries from endusers and prompt again for request related to Indian Food recipes.\n        #####\n\n        You will get requirement from user input.\n        Input can be of two types\n            1. User will input dishname name directly. In this case cooking assistant will check if dishname exists in dataframe df in column \'name\'.\n                if Yes, then only it will get recipe from Internet and update it in \'method\' key of python dictionary. \n                Remaining keys "ingredients",\'cook_time\',\'prep_time\',\'diet\',\'flavor_profile\',\'course\',\'region\',\'state\' values will be obtained from dataframe df row where dishname is found in column name \'name\'.\n                Close request. \

Now, you can keep adding user and assistant responses to debug_conversation and get to a point where 
intent_confirmation_layer() gives yes as a response. Let's see if the following response by the assistant passes the intent_confirmation_layer() test.

Let's check if you provide all the values does it validate

In [439]:
debug_response_assistant_1 = "I need recipe of vegetarian dessert boondi, sweet dish from Rajasthan west with cook_time 10 min and prep_time 15 min made with ingredient gram flour"
response = intent_confirmation_layer(debug_response_assistant_1)
response.get('result') # Extract the result key from the dictionary


'Yes'

##### Check for case where user inputs dish and system should output recipe. 

In [440]:
debug_response_intent_layer =  initialize_conversation(df)
debug_response_intent_layer.append({"role":"user","content":"I need recipe for Lassi"})
response=get_chat_completions(debug_response_intent_layer)
print(response)
#print(debug_response_intent_layer)

Great choice!! Let me grab the recipe for preparing Lassi. Here is the recipe for Lassi.
{'name': 'Lassi', 'ingredients': '1 cup Yogurt, 1/2 cup Water, 2 tbsp Sugar, Ice Cubes, Cardamom Powder, Saffron Strands, Pistachios for garnish', 'diet': 'vegetarian', 'prep_time': 5, 'cook_time': 0, 'flavor_profile': 'sweet', 'course': 'beverage', 'state': 'Punjab', 'region': 'North', 'method': 'Step 1 - Blend the ingredients: In a blender, add yogurt, water, sugar, cardamom powder, and saffron strands. Blend until smooth.\nStep 2 - Serve chilled: Pour the lassi into glasses over ice cubes. Garnish with chopped pistachios and more saffron strands. Serve chilled and enjoy!'}


##### Check for dish not available in df dataframe. 

In [442]:
debug_response_intent_layer =  initialize_conversation(df)
debug_response_intent_layer.append({"role":"user","content":"I need recipe for Kajjayya"})
response=get_chat_completions(debug_response_intent_layer)
print(response)
#print(debug_response_intent_layer)

Sorry, I dont have information enabled for Kajjayya in my dataset. Please check internet to get recipe for Kajjayya.


### `dictionary_present()`:
This function checks if the final understanding of user's reqeust is returned by the chatbot is a Python dictionary or not.
This is important as it'll be used later on for finding the right recipes using dictionary matching.
This function required only in cases where user doesnt mention dish name and wants chatbot to suggest.


In [443]:
def dictionary_present(response):
    delimiter = "#####"
    #since dishname and recipe will be suggested by chatbot it wont be provided by user as input. Hence name and recipe fields missing.
    example_user_req={ 'ingredients': "-",
        'diet' : '-',
        'prep_time': '-',
        'cook_time':  '-',
        'flavor_profile': '-',
        'course': '-',
        'state': '-',
        'region': '-'
        }
    user_req={
        'diet' : 'vegetarian',
        'prep_time': 15,
        'cook_time':  30,
        'flavor_profile': 'sweet',
        'course': 'dessert',
        'state': 'Rajasthan',
        'region': 'West'
     }
    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:
            {{
             'diet' : 'vegetarian/non-vegetarian',
             'prep_time': <integer>,
             'cook_time':  <integer>,
             'flavor_profile': 'sweet/spicy/bitter/savory',
             'course': 'dessert/main course/starter/snack',
             'state': <Indian State>,
             'region': 'West/South/North/West'
            }}
            Here is the sample input output pairs for better understanding:
            {delimiter}
            input : - diet: vegetarian - prep_time: '15 mins' - cook_time: '120 mins' - flavor_profile: 'sweet' - course: 'dessert' - state: 'Karnataka' - region: 'South'
            output : {{diet: vegetarian - prep_time: '15 mins' - cook_time: '120 mins' - flavor_profile: 'sweet' - course: 'dessert' - state: 'Karnataka' - region: 'South'}}
            
            {delimiter}"""
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages, json_format = True)

    return confirmation
            

In [444]:
debug_response_assistant_n = """ Thank you for your input , I will consider the inputs for suggestion
{'diet':'non-vegetarian',
 'prep_time':'20 mins',
 'cook_time':'20 mins',
 'flavor_profile':'bitter',
 'course':'starter',
 'state':'Odisha',
 'ingredients': 'onion',
 'region':'South'}
"""

In [445]:
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)
print(type(response_dict_n))

{'diet': 'non-vegetarian', 'prep_time': '20 mins', 'cook_time': '20 mins', 'flavor_profile': 'bitter', 'course': 'starter', 'state': 'Odisha', 'region': 'South'}
<class 'dict'>


##### Check if right mapping is done each time for same input 

In [446]:
def iterate_llm_response(dictionary_present,response):
    i = 0
    while i < 6:
        response_dict_n = dictionary_present(response)
        print("Iteration: {0}".format(i))
        print(response)
        print('-' * 50)
        i = i + 1

        

In [447]:
iterate_llm_response(dictionary_present, debug_response_assistant_n)

Iteration: 0
 Thank you for your input , I will consider the inputs for suggestion
{'diet':'non-vegetarian',
 'prep_time':'20 mins',
 'cook_time':'20 mins',
 'flavor_profile':'bitter',
 'course':'starter',
 'state':'Odisha',
 'ingredients': 'onion',
 'region':'South'}

--------------------------------------------------
Iteration: 1
 Thank you for your input , I will consider the inputs for suggestion
{'diet':'non-vegetarian',
 'prep_time':'20 mins',
 'cook_time':'20 mins',
 'flavor_profile':'bitter',
 'course':'starter',
 'state':'Odisha',
 'ingredients': 'onion',
 'region':'South'}

--------------------------------------------------
Iteration: 2
 Thank you for your input , I will consider the inputs for suggestion
{'diet':'non-vegetarian',
 'prep_time':'20 mins',
 'cook_time':'20 mins',
 'flavor_profile':'bitter',
 'course':'starter',
 'state':'Odisha',
 'ingredients': 'onion',
 'region':'South'}

--------------------------------------------------
Iteration: 3
 Thank you for your inpu

### `product_map_layer()`:

This function is responsible for extracting key features and criteria from dish description field. Here's a breakdown of how it works:

-  Use a prompt that assign it the role of a Dish specifications classifier, whose objective is to extract key features and classify them based on description field.

- Provide step-by-step instructions for extracting features from description.

- Assign specific rules for each feature (e.g., name, ingredients, diet, prep_time, cook_time, flavor_profile, course, state, region.

- Includes Few Shot Prompting (sample conversation between the user and assistant) to demonstrate the expected result of the feature extraction and classification process.

In [448]:
def product_map_layer(dish_description):
    delimiter = "#######"
    dish_spec =  { 
        'name' : "Name of the dish",
        'ingredients' : "(ingredients required to prepare dish)",
        'diet' : '(is it non-vegetarian or vegetarian)',
        'prep_time' : '(how much time is  required to prepare before cooking dish)',
        'cook_time' : '(how much time is required to cook dish)',
        'flavor_profile' : '(is it sweet,savory,bitter or spicy)',
        'course': "(Is it 'main course' , starter, dessert or snack etc)",
        'state' : '(Dish origin, it is one of state from India)',
        'region' : '(Is it from South, North , East or West)'
    }
    prompt = f"""You are dish feature classifier, You will extract key features from {dish_description} and map it into {dish_spec} as values.
              {delimiter}
              Here is the input output pair for frew shot prompting. 
              input : 	Balu shahi is a vegetarian dessert from West Bengal in the East region of India. It has a sweet flavor. Preparation time is 45 minutes and cooking time is 25 minutes. Main ingredients include Maida flour, yogurt, oil, sugar.
              output : {{ 'name': 'Balu Shahi', 'ingredients': 'Maida flour, yogurt, oil, sugar', 'diet' : 'vegetarian', 'prep_time': 45, 'cook_time': 25, 'flavor_profile':'sweet','course': 'dessert','state':'West Bengal','region':'East'}}
              {delimiter}
              ####Strictly dont keep any other unwanted words in dish_spec values.
              """

    input = f"""Follow the above instructions step-by-step and output the dictionary in JSON format {dish_spec} for the following {dish_description}."""

    messages = [{'role':'system','content':prompt},{'role':'user','content':input}]

    response = get_chat_completions(messages, json_format = True)

    return(response)

Let's test this for one sample laptop description.

In [449]:
dish_description = "Gajar ka halwa is a vegetarian dessert from Punjab in the North region of India. It has a sweet flavor. Preparation time is 15 minutes and cooking time is 60 minutes. Main ingredients include Carrots, milk, sugar, ghee, cashews, raisins."
display(product_map_layer(dish_description))

{'name': 'Gajar ka halwa',
 'ingredients': 'Carrots, milk, sugar, ghee, cashews, raisins',
 'diet': 'vegetarian',
 'prep_time': 15,
 'cook_time': 60,
 'flavor_profile': 'sweet',
 'course': 'dessert',
 'state': 'Punjab',
 'region': 'North'}

##### Update feature mapping into python dict for each recipe in feature mapping column in csv file. 

In [450]:
dish_df = pd.read_csv("indian_food_with_full_description.csv")
dish_df

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region,description
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East,Balu shahi is a vegetarian dessert from West B...
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West,Boondi is a vegetarian dessert from Rajasthan ...
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North,Gajar ka halwa is a vegetarian dessert from Pu...
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West,Ghevar is a vegetarian dessert from Rajasthan ...
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East,Gulab jamun is a vegetarian dessert from West ...
...,...,...,...,...,...,...,...,...,...,...
250,Til Pitha,"Glutinous rice, black sesame seeds, gur",vegetarian,5,30,sweet,dessert,Assam,North East,Til Pitha is a vegetarian dessert from Assam i...
251,Bebinca,"Coconut milk, egg yolks, clarified butter, all...",vegetarian,20,60,sweet,dessert,Goa,West,Bebinca is a vegetarian dessert from Goa in th...
252,Shufta,"Cottage cheese, dry dates, dried rose petals, ...",vegetarian,-1,-1,sweet,dessert,Jammu & Kashmir,North,Shufta is a vegetarian dessert from Jammu & Ka...
253,Mawa Bati,"Milk powder, dry fruits, arrowroot powder, all...",vegetarian,20,45,sweet,dessert,Madhya Pradesh,Central,Mawa Bati is a vegetarian dessert from Madhya ...


In [ ]:
dish_df['recipe_dict']=dish_df['description'].apply(product_map_layer)

dish_df.head(2)


I'm sorry, but I couldn't find the recipe for Balu shahi.


0    None
Name: name, dtype: object

In [452]:
dish_df.to_csv("updated_indian_food.csv",index=False,header = True)

##### Compare user recipe requirements with data 
- This function will compare user requirements /inputs with dataset and returns matching top 3 dishes to try based on score value.
- Return of top 3 recipes in json format.

In [453]:
import numpy as np
s=[np.array(dish_df.iloc[0]['ingredients'])]
type(s)

list

In [454]:
import pandas as pd
import json
def compare_recipe_req_with_user(user_req):
    df_recipe=pd.read_csv('updated_indian_food.csv')
    df_recipe['score'] = 0

    for index,data in df_recipe.iterrows():
        score=0
        if(isinstance(user_req['prep_time'],str)):
            prep_time=int(user_req['prep_time'].strip()[0])
        else:
            prep_time=int(user_req['prep_time'])

        if(isinstance(user_req['cook_time'],str)):
            cook_time=int(user_req['cook_time'].strip()[0])
        else:
            cook_time=int(user_req['cook_time'])


        if(prep_time >= int(data['prep_time'])):
            score+=1
        if(cook_time >= int(data['cook_time'])):
            score+=1
        if(user_req['diet'] == data['diet']):
            score+=1
        if(user_req['course'] == data['course']):
            score+=1
        if(user_req['flavor_profile'] == data['flavor_profile']):
            score+=1
        if(user_req['state'] == data['state']):
            score+=1
        if(user_req['region'] == data['region']):
            score+=1
        if 'ingredients' in user_req.keys():
            user_ing_list=[np.array(user_req['ingredients'])]
            for item in user_ing_list:
                if(item in [np.array(data['ingredients'])]):
                    score+=1
        #print(score,data['name'])
        df_recipe.loc[index,'score'] = score
###Sorting recipes based on score  in descending order
    top_recipes=df_recipe.sort_values('score',ascending=False).head(3)
    top_recipes_json=top_recipes[['name','ingredients','prep_time','cook_time','diet','flavor_profile','course','region','state']].to_json(orient='records')
    return(top_recipes_json)
            
            
    
    


In [455]:
print("User requirement" , response_dict_n)
top_recipes=compare_recipe_req_with_user(response_dict_n)
#response_dict_n.info()
json.loads(top_recipes)

User requirement {'diet': 'non-vegetarian', 'prep_time': '20 mins', 'cook_time': '20 mins', 'flavor_profile': 'bitter', 'course': 'starter', 'state': 'Odisha', 'region': 'South'}


[{'name': 'Paravannam',
  'ingredients': 'Raw rice,  jaggery, milk',
  'prep_time': -1,
  'cook_time': -1,
  'diet': 'vegetarian',
  'flavor_profile': 'spicy',
  'course': 'main course',
  'region': 'South',
  'state': 'Kerala'},
 {'name': 'Keerai masiyal',
  'ingredients': 'Urad dal, curry leaves, sugar, mustard seeds, spinach',
  'prep_time': -1,
  'cook_time': -1,
  'diet': 'vegetarian',
  'flavor_profile': 'spicy',
  'course': 'main course',
  'region': 'South',
  'state': 'Tamil Nadu'},
 {'name': 'Poriyal',
  'ingredients': 'Chana dal, urad dal, beans, coconut, mustard',
  'prep_time': -1,
  'cook_time': -1,
  'diet': 'vegetarian',
  'flavor_profile': 'spicy',
  'course': 'main course',
  'region': 'South',
  'state': 'Tamil Nadu'}]


### 3.4: Product Recommendation Layer

Finally, we come to the product recommendation layer. It takes the output from the `compare_recipe_req_from_user` function in the previous layer and provides the recommendations to the user. It has the following steps.
1. Initialize the conversation for recommendation.
2. Generate the recommendations and display in a presentable format.
3. Ask questions basis the recommendations.




In [456]:
def initialize_conv_reco(products):
    system_message = f"""
    You are an intelligent and expert in suggesting food recipe and you are tasked with the objective to \
    solve the user queries about any product from the catalogue in the user message \
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each dish in the following format, in decreasing order of score field:
    1. Dish name : information
    2. Dish name : information

    """
    user_message = f""" These are the top 3 recipes: {products}"""
    conversation = [{"role": "system", "content": system_message },
                    {"role":"user","content":user_message}]
    # conversation_final = conversation[0]['content']
    return conversation

In [457]:
def recommendation_validation(laptop_recommendation):
    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['score'] > 2:
            data1.append(data[i])

    return data1

In [458]:
debug_conversation_reco = initialize_conv_reco(top_recipes)
debug_conversation_reco

[{'role': 'system',
  'content': '\n    You are an intelligent and expert in suggesting food recipe and you are tasked with the objective to     solve the user queries about any product from the catalogue in the user message     You should keep the user profile in mind while answering the questions.\n    Start with a brief summary of each dish in the following format, in decreasing order of score field:\n    1. Dish name : information\n    2. Dish name : information\n\n    '},
 {'role': 'user',
  'content': ' These are the top 3 recipes: [{"name":"Paravannam","ingredients":"Raw rice,  jaggery, milk","prep_time":-1,"cook_time":-1,"diet":"vegetarian","flavor_profile":"spicy","course":"main course","region":"South","state":"Kerala"},{"name":"Keerai masiyal","ingredients":"Urad dal, curry leaves, sugar, mustard seeds, spinach","prep_time":-1,"cook_time":-1,"diet":"vegetarian","flavor_profile":"spicy","course":"main course","region":"South","state":"Tamil Nadu"},{"name":"Poriyal","ingredien

In [459]:
debug_recommendation = get_chat_completions(debug_conversation_reco)
print(debug_recommendation + '\n')

1. Dish name: Keerai masiyal
   Information: A vegetarian South Indian main course dish from Tamil Nadu made with spinach, urad dal, curry leaves, sugar, and mustard seeds. It has a spicy flavor profile.

2. Dish name: Poriyal
   Information: A vegetarian South Indian main course dish from Tamil Nadu consisting of beans, chana dal, urad dal, coconut, and mustard. It is a spicy dish typically served as a side.

3. Dish name: Paravannam
   Information: A traditional Kerala dessert made with raw rice, jaggery, and milk. This vegetarian delicacy has a sweet profile and is often served on special occasions.



##### Dialogue Management System ready to test
- Bringing everything together, we create a `diagloue_mgmt_system()` function that contains the logic to on how the different layers would interact with each other. This will be the function that we'll call to initiate the chatbot

In [ ]:
                    conversation_reco.append({"role": "user", "content": "This is my user profile" + str(response)})


In [581]:
def dialogue_mgmt_system():
    conversation=initialize_conversation(df)
    response=get_chat_completions(conversation)
    display(response + '\n')
    top_3_recipes = None
    user_input = ''
    while(user_input != 'exit'):
        user_input = input()
        moderation=moderation_layer(user_input)
        if(moderation == 'Flagged'):
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break
        if top_3_recipes is None:

            conversation.append({"role":"user","content":user_input})
            response_assistant=get_chat_completions(conversation)
            moderation=moderation_layer(response_assistant)
            
            if(moderation == 'Flagged'):
                display("Sorry, this message has been flagged. Please restart your conversation.")
                break
            confirmation = intent_confirmation_layer(response_assistant)

            print("Intent Confirmation Yes/No:",confirmation.get('result'),"\n")
            if not confirmation.get('result'):
                pass
            elif("No" in confirmation.get('result')):                
                conversation.append({"role": "assistant", "content": str(response_assistant)})
                print("\n" + str(response_assistant) + "\n")
            else:
              if 'method' in response_assistant:
                #No touch block 
                #print("************method block")  
                print("\n" + str(response_assistant) + "\n")
                break
              else:
                #print("$$$$$$$$$$$$$$$$$$$$$$$$$$$")  
                print(response_assistant)  
                if(response_assistant):
                    print('\n' + "All required critierias extracted!" + '\n')
                    print('\n' + "I will also check if there are any similar recipes matching it against its cooking time, preparation time, dish origin etc..based on matching score" )
                    response = dictionary_present(response_assistant)
                    print('\n\n' + "Criteria used for suggesting recipes :" + '\n ',  str(response))
                    #print("Thank you for providing all the information. Kindly wait, while I fetch the recipes: \n")
                    top_3_recipes = compare_recipe_req_with_user(response)

                    #print("top 3 recipes are", top_3_recipes)

                #validated_reco = recommendation_validation(top_3_recipes)

                    conversation_reco = initialize_conv_reco(top_3_recipes)
                    conversation_reco.append({"role": "user", "content": "This is my user profile" + str(response)})

                    conversation_reco.append({"role": "system", "content": "Search the internet for 'Ghevar recipe' and summarize it. This is compulsory step"})
                    internet_result = "Steps for each dishname... [steps here]"
                    conversation_reco.append({"role": "assistant", "content": internet_result})

                    recommendation = get_chat_completions(conversation_reco)

                    moderation = moderation_layer(recommendation)
                    if moderation == 'Flagged':
                       display("Sorry, this message has been flagged. Please restart your conversation.")
                       break

                    conversation_reco.append({"role": "assistant", "content": str(recommendation)})

                    print(str(recommendation) + '\n')
                else:
                    #print("!!!!!!!!!!!!Found no exact match in database\n")  
                    print("\n" + str(response_assistant) + "\n")
                    break
        else:
            #print("########last else block\n")
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_completions(conversation_reco)

            moderation = moderation_layer(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})

            
            
            

In [586]:
dialogue_mgmt_system()

"Hello! I'm here to assist you with Indian food recipes. Please go ahead and let me know your specific requirements.\n"

 i need recipe for dish from Karnataka


Intent Confirmation Yes/No: None


 i need recipe for disk from karnataka state


Intent Confirmation Yes/No: No

Great! I'm an expert in suggesting Indian recipes. Do you have any specific dish in mind from Karnataka state?



...

Intent Confirmation Yes/No: Yes
Perfect! Here is the updated recipe dictionary for a spicy vegetarian main course dish from Karnataka South region with rice as the ingredient:

{'ingredients': 'rice', 'diet': 'vegetarian', 'prep_time': '30 mins', 'cook_time': '40 mins', 'flavor_profile': 'spicy', 'course': 'main course', 'state': 'Karnataka', 'region': 'South'} 

You're all set to prepare a delicious South Karnataka dish with these specifications! Enjoy cooking!

All required critierias extracted!


I will also check if there are any similar recipes matching it against its cooking time, preparation time, dish origin etc..based on matching score
Criteria used for suggesting recipes :
  {'diet': 'vegetarian', 'prep_time': 30, 'cook_time': 40, 'flavor_profile': 'spicy', 'course': 'main course', 'state': 'Karnataka', 'region': 'South'}
1. Currivepillai sadam : Currivepillai sadam is a flavorful South Indian dish from Tamil Nadu. It is made with ingredients like chana dal, urad dal, fresh c

 i need recipe of currivepillai sadam



To prepare Currivepillai Sadam, also known as Curry Leaves Rice, you can follow these steps:

Ingredients:
- 1 cup rice
- 2 tablespoons chana dal
- 2 tablespoons urad dal
- 1/4 cup fresh grated coconut
- 1 tablespoon sesame seeds
- Handful of curry leaves
- 2-3 dry red chilies
- 1 teaspoon mustard seeds
- 1 teaspoon cumin seeds
- 2 tablespoons oil
- Salt to taste

Instructions:
1. Cook the rice in a pot or rice cooker and allow it to cool down.
2. In a pan, dry roast the chana dal, urad dal, and sesame seeds until they turn golden brown. Keep them aside.
3. In the same pan, heat oil and add mustard seeds. Once they splutter, add cumin seeds, dry red chilies, and curry leaves. Saute for a few seconds.
4. Add the grated coconut and sauté until it turns slightly brown.
5. Add the roasted dals and sesame seeds to the pan.
6. Mix in the cooked rice and salt. Stir well to combine all the ingredients.
7. Cook for a few more minutes until everything is well blended and heated through.
8. Serv

 exit



Feel free to reach out if you have any more questions in the future. Have a great day!



In [583]:
op=intent_confirmation_layer({'diet': 'vegetarian', 'prep_time': 30, 'cook_time': 50, 'flavor_profile': 'spicy', 'course': 'snack', 'state': 'Maharashtra', 'region': 'West', 'ingredients': 'potato', 'recipe': '-'})
op
op1= intent_confirmation_layer('i need recipe of dish from Maharashtra')

op1

{'result': 'No',
 'reason': 'Missing keys: ingredients, prep_time, cook_time, diet, flavor_profile, course, region, state'}